# Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader

# Data

In [ ]:
INPUT_SIZE = 1
SEQ_LENGTH = 5
HIDDEN_SIZE = 2
NUM_LAYERS = 1
BATCH_SIZE = 4
NUM_EPOCHS = 20000
LEARNING_RATE = 0.01

data = torch.Tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
print("Data:", data.shape, "\n\n", data)

Data: torch.Size([20]) 

 tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
        15., 16., 17., 18., 19., 20.])


In [72]:
# Chunck the data into 15 chuncks of length 5
chunks = []
targets = []
for i in range(len(data)-SEQ_LENGTH):
    chunks.append(data[i:(i+SEQ_LENGTH)])
    targets.append(data[i+SEQ_LENGTH])



X = torch.stack(chunks).unsqueeze(-1)
Y = torch.stack(targets).unsqueeze(-1)
print("X:", X.shape, "\n\n", X)
print("y:", Y.shape, "\n\n", Y)

X: torch.Size([15, 5, 1]) 

 tensor([[[ 1.],
         [ 2.],
         [ 3.],
         [ 4.],
         [ 5.]],

        [[ 2.],
         [ 3.],
         [ 4.],
         [ 5.],
         [ 6.]],

        [[ 3.],
         [ 4.],
         [ 5.],
         [ 6.],
         [ 7.]],

        [[ 4.],
         [ 5.],
         [ 6.],
         [ 7.],
         [ 8.]],

        [[ 5.],
         [ 6.],
         [ 7.],
         [ 8.],
         [ 9.]],

        [[ 6.],
         [ 7.],
         [ 8.],
         [ 9.],
         [10.]],

        [[ 7.],
         [ 8.],
         [ 9.],
         [10.],
         [11.]],

        [[ 8.],
         [ 9.],
         [10.],
         [11.],
         [12.]],

        [[ 9.],
         [10.],
         [11.],
         [12.],
         [13.]],

        [[10.],
         [11.],
         [12.],
         [13.],
         [14.]],

        [[11.],
         [12.],
         [13.],
         [14.],
         [15.]],

        [[12.],
         [13.],
         [14.],
         [15.],
     

# RNN

In [76]:
rnn = nn.LSTM(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers =
1, batch_first=True)

inputs = data.view(BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE)
out, h_n = rnn(inputs)

fc = nn.Linear(HIDDEN_SIZE, 1)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(list(rnn.parameters()) + list(fc.parameters()), lr=LEARNING_RATE)



# Training loop

In [77]:
# Training loop
for epoch in range(NUM_EPOCHS):
    # Forward pass
    out, _ = rnn(X)                # out: (batch, seq_len, hidden)
    last_hidden = out[:, -1, :]    # use the last time step
    prediction = fc(last_hidden)   # map to output
 
    # Compute loss
    loss = criterion(prediction, Y)
 
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
 
    if (epoch+1) % 20 == 0:
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {loss.item():.4f}")


Epoch 20/1000, Loss: 192.3123
Epoch 40/1000, Loss: 175.9007
Epoch 60/1000, Loss: 146.4844
Epoch 80/1000, Loss: 129.6814
Epoch 100/1000, Loss: 115.7928
Epoch 120/1000, Loss: 103.8293
Epoch 140/1000, Loss: 93.3349
Epoch 160/1000, Loss: 84.0482
Epoch 180/1000, Loss: 75.7985
Epoch 200/1000, Loss: 68.4616
Epoch 220/1000, Loss: 61.9394
Epoch 240/1000, Loss: 56.1497
Epoch 260/1000, Loss: 51.0215
Epoch 280/1000, Loss: 46.4911
Epoch 300/1000, Loss: 42.5007
Epoch 320/1000, Loss: 38.9971
Epoch 340/1000, Loss: 35.9308
Epoch 360/1000, Loss: 33.2542
Epoch 380/1000, Loss: 30.7835
Epoch 400/1000, Loss: 27.3650
Epoch 420/1000, Loss: 24.8820
Epoch 440/1000, Loss: 22.6994
Epoch 460/1000, Loss: 20.7293
Epoch 480/1000, Loss: 18.9481
Epoch 500/1000, Loss: 17.3356
Epoch 520/1000, Loss: 15.8747
Epoch 540/1000, Loss: 14.5501
Epoch 560/1000, Loss: 13.3478
Epoch 580/1000, Loss: 12.2554
Epoch 600/1000, Loss: 11.2615
Epoch 620/1000, Loss: 10.3547
Epoch 640/1000, Loss: 9.5272
Epoch 660/1000, Loss: 8.7726
Epoch 680/

# Test Prediction

In [79]:
#  Test prediction
test_seq = torch.tensor([16.0, 17.0, 18.0, 19.0, 20.0]).view(1, SEQ_LENGTH, 1)
out, _ = rnn(test_seq)
predicted = fc(out[:, -1, :])
print(f"\nPrediction for [16, 17, 18, 19, 20] → {predicted.item():.2f}")



Prediction for [16, 17, 18, 19, 20] → 15.74
